In [4]:
import numpy as np
from scipy import stats

### НАЧНЁМ С ОДНОВЫБОРОЧНЫХ ТЕСТОВ

Помним, что данные должны приходить из нормального распределения. Сгенерируем множество точек из нормального распределения (для воспроизводимости зафиксируем random state) со средним, равным 5, и стандартным отклонением, равным 2.

In [5]:
np.random.seed(13)
random_normal = np.random.normal(5, 2, 100)
random_bin = np.random.choice([0, 1], size=(100,), p=[0.8, 0.2])

Проверим гипотезу, что среднее выборки, сгенерированной из нормального распределения, равно 0:

In [8]:
stats.ttest_1samp(random_normal, 0.0)

TtestResult(statistic=27.315846581411247, pvalue=6.762242033211967e-48, df=99)

По результату мы видим, что p_value= 6.762242033211871e-48, соответственно, гипотеза о равенстве среднего нулю отвергается на уровне значимости 0.01 (вообще говоря, почти на любом уровне значимости).

Теперь проверим против 5.0:

In [9]:
stats.ttest_1samp(random_normal, 5.0)

TtestResult(statistic=0.6236095710595042, pvalue=0.5343182132984923, df=99)

Мы видим, что гипотеза о равенстве 5 принимается с p_value=0.53.

Проведём тест на равенство доли единиц во второй выборке 0.5:

In [14]:
num_successes = sum(random_bin)
total_trials = len(random_bin)

# Perform the binomial test
stats.binomtest(k=num_successes, n=total_trials, p=0.5)


BinomTestResult(k=20, n=100, alternative='two-sided', statistic=0.2, pvalue=1.1159089057251951e-09)

Заметим, что k = [число позитивных действий, число фейлов]. Видим, что гипотеза отвергается практически на любом уровне значимости с p_value=1.115908905725195e-09.

---

### А ТЕПЕРЬ — ДВУХВЫБОРОЧНЫЙ ТЕСТ!

Мы увидели, что проводить статистический тест со SciPy просто! Давайте теперь проделаем то же самое со множественной проверкой гипотез.

Сгенерируем множества точек:

In [15]:
random_normal_5 = np.random.normal(5, 2, 100)
random_normal_false = np.random.normal(7, 2, 100)

Проверим двухвыборочными тестами:

In [17]:
print(stats.ttest_ind(random_normal_5, random_normal))
print(stats.ttest_ind(random_normal_false, random_normal))

TtestResult(statistic=-1.109776693013421, pvalue=0.2684412714449198, df=198.0)
TtestResult(statistic=7.56418694452888, pvalue=1.4333446457467145e-12, df=198.0)


Видим, что в первом случае нулевая гипотеза не отвергается, а во втором — отвергается. Что ж, мы молодцы, и самое время перейти к задачам, не так ли?

Стоп! А как же тест на равенство пропорций (долей)? К сожалению, в SciPy он не реализован, поэтому придётся обратиться к другому пакету — statsmodels.

In [18]:
from statsmodels.stats.proportion import proportions_ztest

In [19]:
random_bin_2 = np.random.choice([0, 1], size=(100,), p=[0.8, 0.2])
random_bin_false = np.random.choice([0, 1], size=(100,), p=[0.6, 0.4])

Посмотрим на результат применения теста:

In [20]:
print(
    proportions_ztest(
        count=[sum(random_bin), sum(random_bin_2)], # количество “успехов” в каждой из выборок. 
        nobs=[len(random_bin), len(random_bin_2)], # общее количество наблюдений в каждой выборке.
    )
)

print(
    proportions_ztest(
        count=[sum(random_bin), sum(random_bin_false)], # количество “успехов” в каждой из выборок. 
        nobs=[len(random_bin), len(random_bin_false)], # общее количество наблюдений в каждой выборке.
    )
)

(-0.5163595320566151, 0.6056033242983083)
(-2.5197631533948477, 0.011743382301172597)


Первое число — значение статистики, второе — p_value. Видим, что результаты совпадают с ожидаемыми.

In [26]:
# Задание 5.2
# Какое значение p_value * 10**7 получается при проведении теста равенства доли 0.1 по выборке, 
# сгенерированной кодом ниже (округлите до двух знаков после запятой)?

np.random.seed(13)
random_bin = np.random.choice([0, 1], size=(100,), p=[0.7, 0.3])

stats.binomtest(k=sum(random_bin), n=len(random_bin), p=0.1).pvalue

3.481426687624852e-07

In [29]:
# Задание 5.3
# Какое значение статистики получается при проведении теста равенства средних 3 по выборке, 
# сгенерированной кодом ниже (округлите до двух знаков после запятой)?

np.random.seed(21)
random_normal = np.random.normal(18, 20, 100)

stats.ttest_1samp(random_normal, 3).statistic

7.974042103707906

In [34]:
# Задание 5.4
# На каких уровнях значимости отвергается гипотеза равенства среднего 7 по выборке, сгенерированной кодом ниже:

np.random.seed(21)
random_normal = np.random.normal(18, 20, 10)

# Проведем t-тест для каждого уровня значимости
for alpha in [0.05, 0.01, 0.1, 0.15]:
    t_stat, p_value = stats.ttest_1samp(random_normal, 7)
    
    if p_value < alpha:
        print(f"rejected on {alpha} level")
    else:
        pass

rejected on 0.15 level
